In [32]:
#These lines set up inline plotting, and apply a standard size
%matplotlib inline
import matplotlib
matplotlib.rc('font', **{'size': 22})
# Standard includes
import datetime
import re
import os
import json
from elasticsearch import Elasticsearch
from elasticsearch.helpers import scan
import matplotlib.pyplot as plt
from time import time
import timeit
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
from matplotlib.dates import HourLocator

In [33]:
infilesizes = {}
if os.path.isfile("ART_filesizes.txt"):
    lines = [line.strip('\n') for line in open("ART_filesizes.txt")]
    for l in lines:
        infilesizes[l.split(',')[0]] = l.split(',')[1]

if not 'mcFileBPHY8' in infilesizes.keys(): infilesizes['mcFileBPHY8'] = os.path.getsize("/cvmfs/atlas-nightlies.cern.ch/repo/data/data-art/DerivationFrameworkART/AOD.11705353._000001.pool.root.1")
if not 'mcFileEXOT23' in infilesizes.keys():infilesizes['mcFileEXOT23'] = os.path.getsize("/cvmfs/atlas-nightlies.cern.ch/repo/data/data-art/DerivationFrameworkART/user.ctreado.11717804.EXT0._000056.DAOD_RPVLL.pool.root")
if not 'mcFile' in infilesizes.keys():infilesizes['mcFile'] = os.path.getsize("/cvmfs/atlas-nightlies.cern.ch/repo/data/data-art/DerivationFrameworkART/AOD.12169019._004055.pool.root.1")
#if not 'dataFile' in infilesizes.keys():infilesizes['dataFile'] = os.path.getsize("/cvmfs/atlas-nightlies.cern.ch/repo/data/data-art/DerivationFrameworkART/AOD.12818484._004694.pool.root.1")
if not 'dataFile' in infilesizes.keys(): infilesizes['dataFile'] = os.path.getsize("/cvmfs/atlas-nightlies.cern.ch/repo/data/data-art/DerivationFrameworkART/data18_13TeV.00348403.physics_Main.merge.AOD.f920_m1947._lb0829._0001.1")
if not 'dataFileEXOT23' in infilesizes.keys():infilesizes['dataFileEXOT23'] = os.path.getsize("/cvmfs/atlas-nightlies.cern.ch/repo/data/data-art/DerivationFrameworkART/DAOD_RPVLL.09788089._000056.pool.root.1")
if not 'dataFileDelayed' in infilesizes.keys():infilesizes['dataFileDelayed'] = os.path.getsize("/cvmfs/atlas-nightlies.cern.ch/repo/data/data-art/DerivationFrameworkART/AOD.11270451._000007.pool.root.1")
if not 'dataFileBLS' in infilesizes.keys():infilesizes['dataFileBLS'] = os.path.getsize("/cvmfs/atlas-nightlies.cern.ch/repo/data/data-art/DerivationFrameworkART/data17_13TeV.00337491.physics_BphysLS.merge.AOD.f873_m1885._lb0100._0001.1")
if not 'heavyIonFile' in infilesizes.keys():infilesizes['heavyIonFile'] = os.path.getsize("/cvmfs/atlas-nightlies.cern.ch/repo/data/data-art/DerivationFrameworkART/data17_hi.00338037.physics_MinBias.merge.AOD.f900_m1912._lb0292._0009.1")
if not 'truthFile' in infilesizes.keys():infilesizes['truthFile'] = os.path.getsize("/cvmfs/atlas-nightlies.cern.ch/repo/data/data-art/DerivationFrameworkART/EVNT.05192704._020091.pool.root.1")
    
infile = open("ART_filesizes.txt","w")
for key in infilesizes.keys():
    infile.write("%s,%i\n"%(key,int(infilesizes[key])))
infile.close()

In [34]:
def getFormat(deriv):
    format = ''.join([i for i in deriv if not i.isdigit()])
    if "_" in format: format = format.split("_")[-1]
    return format

In [35]:
def parseARTsetupfile(dic,date_str,vb=0):
    trains = {}
    for key in dic.keys():
        if vb: print("KEY = ",key)
        ln = key
        if vb: print("ln ",ln)
        if ".sh" in ln:
            sp = (ln.split(".")[0]).split("_")
            if vb: print("here",ln)
            if "DELAYED" in sp[1] or "BPHYSLS" in sp[1]:
                regexp = re.search('(\D{7})(\D{4,}\d{1,2})', sp[1])
                formatOne = regexp.group(2)
                scope     = regexp.group(1)
            else:
                regexp = re.search('(\D{2,4}\d{2})(\D{4,}.*)', sp[1])
                try:
                    formatOne = regexp.group(2)
                    scope     = regexp.group(1)
                except:
                    print("WARNING \t Could not find train or scope for %s" %ln)
                    continue
            train = " ".join([formatOne] + sp[2:])
            if vb: print("1",train)
            if len(dic[key][date_str]['jobs']) > 1:
                print("ERROR \t More than one job!")
            jobid  = dic[key][date_str]['jobs'][0]['origpandaid']#.keys())[0]
            if vb: print("jobid",jobid)
            status = dic[key][date_str]['jobs'][0]['jobstatus']
            if vb: print("status",status)
            if not jobid in trains.keys():
                trains[jobid] = {"scope":scope,"status":status,"cars":train}
    if vb: print("Return",trains)
    return trains
    

In [36]:
def readJSONfile(jsonfilename,date_str,vb=0):
    with open(jsonfilename, 'r') as f:
         data = json.load(f)
    try:
        trains = parseARTsetupfile(data["artjobs"]["DerivationFrameworkART"]["21.2/AthDerivation/x86_64-slc6-gcc62-opt"],date_str)   
    except:
        print("Problem decoding ",data)
        return 0
    return trains

In [37]:
def readCURLfile(curlfilename, vb = 0):
    lines  = [line.strip() for line in open(curlfilename)]
    trains = {}
    for i in range(len(lines)):
        #print(lines[i])
        ln = lines[i].strip()
        if ".sh" in ln: 
            sp = (ln.split(".")[0]).split("_")
            print("here",ln)
            if "DELAYED" in sp[1] or "BPHYSLS" in sp[1]:
                regexp = re.search('(\D{7})(\D{4,}\d{1,2})', sp[1])
                formatOne = regexp.group(2)
                scope     = regexp.group(1)
            else:
                regexp = re.search('(\D{2,4}\d{2})(\D{4,}.*)', sp[1])
                try:
                    formatOne = regexp.group(2)
                    scope     = regexp.group(1)
                except:
                    print("WARNING \t Could not find train or scope for %s" %ln)
                    continue
            train = " ".join([formatOne] + sp[2:])
            if vb: print(train)
            j = i+1
            while ".sh" not in lines[j] and j < (len(lines)-1):
                #print(lines[j])
                if "_job" in lines[j]:
                    try:
                        status = re.search('.*_job\">\s*(\w*)\s*</div>', lines[j]).group(1)
                        if vb: print(status)
                        #trains[train][scope]["status"] = status
                    except:
                        print("WARNING \t Could not find status of line %s" %line[j])
                        continue
                if "pandaid" in lines[j]:
                    try:
                        jobid = re.search('.*pandaid=(\d{10})\">\d{10}</a>', lines[j]).group(1)
                        if vb: print(jobid)
                        #trains[train][scope]["jobid"] = jobid
                    except:
                        print("WARNING \t Could not find jobid of line %s" %lines[j])
                        continue
                j += 1
                
            if not jobid in trains.keys():
                trains[jobid] = {"scope":scope,"status":status,"cars":train}


    return trains

In [38]:
def sortTwoNParrays(array1, array2):
    evfrac_sorted  = np.empty((0))
    formats_sorted = np.empty((0))
    while array1.shape[0] > 0:
        i = 0
        max_ef = -9999
        max_id = -9999
        for ef in np.nditer(array1):
            if ef > max_ef:
                max_ef = ef
                max_id = i
            i += 1
        evfrac_sorted = np.append(evfrac_sorted,[max_ef],axis=0)
        formats_sorted = np.append(formats_sorted,[formats[max_id]],axis=0)
        array1 = np.delete(array1,max_id,0)
        array2 = np.delete(array2,max_id,0)
    return evfrac_sorted, formats_sorted

In [39]:
#today = datetime.today() - timedelta(days=31)
## need date on format yyyy-mm-dd
#date_str = today.strftime('%Y-%m-%d')
#print("Looking at ART tests from %s" %date_str)
#cmd = "curl --silent -k -b bigpanda.cookie.txt -H \'Accept: application/json\' -H \'Content-Type: application/json\' \"https://bigpanda.cern.ch/art/jobs/?package=DerivationFrameworkART&branch=21.2/x86_64-slc6-gcc62-opt/AthDerivation&ntag=%s&view=packages&json\" &> ut.txt" %(date_str)
#os.system(cmd)

In [40]:
#train = readCURLfile("./ut.txt")

In [41]:
#trainsAndIDs = {}
#for ids in train.keys():
#    t = train[ids]["cars"]
#    s = train[ids]["scope"]
#    if not s in trainsAndIDs.keys():
#        trainsAndIDs[s] = {}
#    trainsAndIDs[s][t] = {}
#for key in sorted(trainsAndIDs.keys()):
#    #print(key+":")
#    nt = 1
#    for t in sorted(trainsAndIDs[key].keys()):
#        #print("%03i) %s" %(nt,t))
#        nt += 1

In [42]:
quantities = [
    'cpuconsumptiontime',
    'nevents',
    'durationsec',
    'outputfilebytes'
]


In [43]:
# Execute the queries
es = Elasticsearch(['atlas-kibana-dev.mwt2.org'],timeout=120)

In [44]:
for i in range(1,0,-1):
    today = datetime.today() - timedelta(days=i)
    # need date on format yyyy-mm-dd
    date_str = today.strftime('%Y-%m-%d')
    print("Looking at ART tests from %s" %date_str)
    cmd = "curl --silent -k -b bigpanda.cookie.txt -H \'Accept: application/json\' -H \'Content-Type: application/json\' \"https://bigpanda.cern.ch/art/jobs/?package=DerivationFrameworkART&branch=21.2/AthDerivation/x86_64-slc6-gcc62-opt&ntag=%s&view=packages&json\" &> ut.txt" %(date_str)
    os.system(cmd)

    train = readJSONfile("./ut.txt",date_str)
    
    if type(train) is int:
        print("WARNING \t Empty json for %s" %date_str)
        continue
    
    trainsAndIDs = {}
    for ids in train.keys():
        t = train[ids]["cars"]
        s = train[ids]["scope"]
        if not s in trainsAndIDs.keys():
            trainsAndIDs[s] = {}
        trainsAndIDs[s][t] = {}
    for key in sorted(trainsAndIDs.keys()):
        #print(key+":")
        nt = 1
        for t in sorted(trainsAndIDs[key].keys()):
            #print("%03i) %s" %(nt,t))
            nt += 1

    # Set up query for the jobs relevant to the trains
    start_time = timeit.default_timer()
    taskCounter = 0
    jobIndex = "jobs_archive_2018*"

    to_read = quantities
    to_read.append('pandaid')
    to_read.append('maxpss')
    to_read.append('maxvmem')
    to_read.append('jobstatus')
    nid = 0
    for theId in train.keys():
        jobQuery = {
            "_source": to_read,
            "query": {
                "bool": {
                    "must": [
                        {"term":{"pandaid": theId}}
                    ]
                }
            }
        }
        # query the jobs
        jobs = scan(es, query=jobQuery, index=jobIndex, scroll='5m', timeout="5m", size=1000)
        for res in jobs: # Loop over jobs from that task
            #print("Train %s on %s" %(train[theId]["cars"],train[theId]["scope"]))
            for quantity in quantities: # get the relevant quantities
                if quantity in res['_source'].keys():
                    trainsAndIDs[train[theId]["scope"]][train[theId]["cars"]][quantity] = res['_source'][quantity]
                    #print(quantity," = " ,res['_source'][quantity])
        nid += 1
        #if nid > 10: break
        
    metrics = {}
    for tr in to_read:
        metrics[tr] = {}
        for scope in trainsAndIDs.keys():
            if os.path.isfile("data/%s_timeseries_%s.csv"%(tr,scope)):
                metrics[tr][scope] = pd.read_csv("data/%s_timeseries_%s.csv"%(tr,scope))
                metrics[tr][scope] = metrics[tr][scope].loc[:,~metrics[tr][scope].columns.duplicated()]
            else:
                for scope in trainsAndIDs.keys():
                    metrics[tr][scope] = pd.DataFrame(columns=['date']+sorted(trainsAndIDs[scope].keys()))
                    
                    
    for met in metrics.keys():
        for scope in trainsAndIDs.keys():
            idx_num = -999
            if not date_str in np.array(metrics[met][scope]['date'], dtype=object):
                metrics[met][scope].append(pd.DataFrame(columns=['date']+sorted(trainsAndIDs[scope].keys())))
                idx_num = len(metrics[met][scope].index)
            else:
                for i in range(0,metrics[met][scope].shape[0]):
                    if metrics[met][scope].iloc[i]['date'] == date_str:
                        #print("Found %s already inside on index %i" %(date_str,i))
                        idx_num = i
                        break
            if idx_num == -999: 
                print("No entry found")
                continue
            metrics[met][scope].loc[idx_num,'date'] = date_str
            df = []
            n = 0
            for configs in sorted(trainsAndIDs[scope].keys()):
                if met in trainsAndIDs[scope][configs].keys():   
                    metrics[met][scope].loc[idx_num,configs] = trainsAndIDs[scope][configs][met]
            metrics[met][scope].to_csv("data/%s_timeseries_%s.csv"%(met,scope),index=False)


Looking at ART tests from 2018-08-13


In [45]:
print(len(metrics['jobstatus']['mc16'].index))
print(metrics['jobstatus']['mc16'].shape)
for i in range(0,metrics['jobstatus']['mc16'].shape[0]):
    print(metrics['jobstatus']['mc16'].iloc[i]['date'])

30
(30, 163)
2018-04-28
2018-04-29
2018-04-25
2018-04-26
2018-04-27
2018-04-30
2018-05-01
2018-05-02
2018-05-03
2018-05-06
2018-05-04
2018-05-05
2018-05-07
2018-05-08
2018-05-09
2018-05-11
2018-05-12
2018-05-13
2018-05-14
2018-05-15
2018-05-16
2018-05-17
2018-05-18
2018-05-19
2018-05-20
2018-05-21
2018-05-22
2018-05-23
2018-05-24
2018-08-13


In [16]:
evfrac  = np.empty((0))
formats = np.empty((0))
print(evfrac.shape)
for configs in trainsAndIDs["mc16"].keys():
    if len(configs.split()) > 1: continue
    if 'nevents' in trainsAndIDs["mc16"][configs].keys(): 
        #print(float(trainsAndIDs["data17"][configs]["nevents"])/500.)
        evfrac = np.append(evfrac,[float(trainsAndIDs["mc16"][configs]["nevents"])/500.],axis=0)
        formats = np.append(formats,[configs],axis=0)
evfrac_sorted, formats_sorted = sortTwoNParrays(evfrac, formats)

(0,)


In [17]:
#i = 0
#for ef in np.nditer(evfrac_sorted):
#    print("%15s : %10.2f" %(formats_sorted[i],ef))
#    i += 1

In [18]:
#plt.figure(figsize=(220,40))
#plt.bar(np.arange(evfrac_sorted.shape[0]),evfrac_sorted,tick_label=formats_sorted,width=0.8)
#plt.xlabel('xlabel', fontsize=100)
#plt.xticks(rotation='vertical',fontsize=120)

In [19]:
#metrics = {}
#for tr in to_read:
#    metrics[tr] = {}
#    for scope in trainsAndIDs.keys():
#        if os.path.isfile("data/%s_timeseries_%s.csv"%(tr,scope)):
#            metrics[tr][scope] = pd.read_csv("data/%s_timeseries_%s.csv"%(tr,scope))
#        else:
#            for scope in trainsAndIDs.keys():
#                metrics[tr][scope] = pd.DataFrame(columns=['date']+sorted(trainsAndIDs[scope].keys()))

In [20]:
#for met in metrics.keys():
#    for scope in trainsAndIDs.keys():
#        if not date_str in np.array(metrics[met][scope]['date'], dtype=object):
#            metrics[met][scope].append(pd.DataFrame(columns=['date']+sorted(trainsAndIDs[scope].keys())))
#        else:
#            continue
#        idx_num = len(metrics[met][scope].index)
#        metrics[met][scope].loc[idx_num,'date'] = date_str
#        df = []
#        n = 0
#        for configs in sorted(trainsAndIDs[scope].keys()):
#            if met in trainsAndIDs[scope][configs].keys():   
#                metrics[met][scope].loc[idx_num,configs] = trainsAndIDs[scope][configs][met]
#        metrics[met][scope].to_csv("data/%s_timeseries_%s.csv"%(met,scope),index=False)

In [48]:
group_trains = {}
for scope in trainsAndIDs.keys():
    group_trains[scope] = {}
    for configs in sorted(trainsAndIDs[scope].keys()):
        if len(configs.split()) == 1:
            grp = getFormat(configs)
            if not grp in group_trains[scope].keys():
                group_trains[scope][grp] = []
        else:
            grp = 'train'
            if not grp in group_trains[scope].keys():
                group_trains[scope][grp] = []
        group_trains[scope][grp].append(configs)

In [49]:
print(group_trains)
scope = "data17"
for grp in group_trains[scope].keys():
    for configs in group_trains[scope][grp]:
        print(configs)

{'mc15': {'TRUTH': ['TRUTH0', 'TRUTH1', 'TRUTH3']}, 'mc16': {'EXOT': ['EXOT0', 'EXOT10', 'EXOT12', 'EXOT13', 'EXOT15', 'EXOT17', 'EXOT19', 'EXOT2', 'EXOT20', 'EXOT21', 'EXOT22', 'EXOT23', 'EXOT24', 'EXOT25', 'EXOT26', 'EXOT3', 'EXOT4', 'EXOT5', 'EXOT6', 'EXOT7', 'EXOT8', 'EXOT9'], 'train': ['TOPQ1 TOPQ2 TOPQ4 TOPQ5'], 'JETM': ['JETM1', 'JETM10', 'JETM12', 'JETM13', 'JETM4', 'JETM5', 'JETM6', 'JETM7', 'JETM8', 'JETM9'], 'PHYSVAL': ['PHYSVAL'], 'STDM': ['STDM2', 'STDM3', 'STDM4', 'STDM5', 'STDM6', 'STDM7', 'STDM8', 'STDM9'], 'TAUP': ['TAUP1', 'TAUP2', 'TAUP3', 'TAUP4'], 'EGAM': ['EGAM1', 'EGAM2', 'EGAM3', 'EGAM4', 'EGAM5', 'EGAM6', 'EGAM7', 'EGAM8', 'EGAM9'], 'SUSY': ['SUSY1', 'SUSY10', 'SUSY11', 'SUSY12', 'SUSY15', 'SUSY16', 'SUSY17', 'SUSY2', 'SUSY3', 'SUSY4', 'SUSY5', 'SUSY6', 'SUSY7', 'SUSY8', 'SUSY9'], 'IDTR': ['IDTR1'], 'HIGGD': ['HIGG1D1', 'HIGG1D2', 'HIGG2D1', 'HIGG2D4', 'HIGG2D5', 'HIGG3D1', 'HIGG3D3', 'HIGG4D1', 'HIGG4D2', 'HIGG4D3', 'HIGG4D4', 'HIGG4D5', 'HIGG4D6', 'HIGG5D1', 

KeyError: 'data17'

In [50]:

%%capture

#print(infilesizes)
#for scope in trainsAndIDs.keys():
#    print("%s : %s" %(scope,",".join(trainsAndIDs[scope].keys())))
#to_read


In [24]:
ylabels = {"maxpss":"maxpss [MB]",
           "maxvmem":"maxvmem [MB]",
           "outputfilebytes":"size frac. [%]",
           "nevents":"skim frac. [%]",
           "durationsec":"duration [sec.]",
           "cpuconsumptiontime":"cpu time [sec.]"}
tables_info = {}
for metric in to_read:
    if metric == 'pandaid': continue
    if metric == 'durationsec': continue
    if metric == 'jobstatus': continue
    toplot = {}
    tables_info[metric] = {}
    #metric = "outputfilebytes"
    print(metric)
    for scope in trainsAndIDs.keys():
        #print(scope)
        if os.path.isfile("data/%s_timeseries_%s.csv"%(metric,scope)):
            toplot[scope] = pd.read_csv("data/%s_timeseries_%s.csv"%(metric,scope),index_col='date',parse_dates=['date'])
        #scope = 'data17'
        if metric == 'nevents': toplot[scope].loc[:,:] /= 500
        elif metric in ['maxpss','maxvmem']: toplot[scope].loc[:,:] *= 0.001
        elif metric == 'outputfilebytes':
            insize = 1.0
            if 'data17' in scope or 'data18' in scope:
                toplot[scope].loc[:,'EXOT23'] /= float(infilesizes['dataFileEXOT23'])
                toplot[scope].loc[:,group_trains[scope]['HION']] /= float(infilesizes['heavyIonFile'])
                toplot[scope][toplot[scope].columns.difference(group_trains[scope]['HION']+['EXOT23'])] /= float(infilesizes['dataFile'])
            elif 'mc16' in scope:
                toplot[scope].loc[:,'EXOT23'] /= float(infilesizes['mcFileEXOT23'])
                toplot[scope].loc[:,'BPHY8'] /= float(infilesizes['mcFileBPHY8'])
                toplot[scope][toplot[scope].columns.difference(['BPHY8','EXOT23'])] /= float(infilesizes['mcFile'])
            elif 'DELAYED' in scope:
                toplot[scope].loc[:,:] /= float(infilesizes['dataFileDelayed'])
            elif 'BPHYSLS' in scope:
                toplot[scope].loc[:,:] /= float(infilesizes['dataFileBLS'])
            elif 'mc15' in scope:
                toplot[scope].loc[:,:] /= float(infilesizes['truthFile'])
            else:
                print("WARNING \t Scope %s does not have any file connected to it" %scope)
            toplot[scope].loc[:,:] *= 100.
        ngrp = 0
        for grp in group_trains[scope]:
            ngrp += 1
            print("Doing train %i/%i for %s and %s"%(ngrp,len(group_trains[scope]),scope,metric))
            try:
                ax = toplot[scope].plot(y=group_trains[scope][grp],figsize=(40,15),legend=False,
                                        marker='o',title="Group: %s, Scope: %s" %(grp,scope),colormap='rainbow',
                                        markersize=20)
            except:
                print("WARNING \t For metric %s: could not plot %s for group %s with formats %s" %(metric,scope,grp,",".join(group_trains[scope][grp])))
                continue
            ax.xaxis_date()
            #plt.ylabel('maxvmem [MB]')
            if not 'train' in grp: ax.legend(bbox_to_anchor=(1.01, 1.05))
            else: ax.legend(loc=9, bbox_to_anchor=(0.5, -0.2),ncol=3)
            #if 'cpuconsumptiontime' in metric: 
            #    ax.yaxis.set_major_locator(HourLocator())
            #    ax.yaxis.set_major_formatter(DateFormatter('%H:%M'))
            #    plt.gca().yaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M')) 
            ax.set_xticks(np.array(metrics[metric][scope]['date'], dtype=object))
            plt.ylabel(ylabels[metric])
            plt.xlabel("ART date")
            #print("Saving figure to plots/%s_%s_%s.png" %(metric,scope,grp))
            ax.get_figure().savefig("plots/%s_%s_%s.png" %(metric,scope,grp))
            plt.clf()
            #break
    tables_info[metric] = toplot.copy()

        #break
    #break

cpuconsumptiontime
Doing train 1/1 for mc15 and cpuconsumptiontime


/usr/lib64/python3.4/site-packages/pandas/plotting/_core.py:1716: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  series.name = label


Doing train 1/15 for mc16 and cpuconsumptiontime


/usr/lib64/python3.4/site-packages/pandas/plotting/_core.py:1716: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  series.name = label


Doing train 2/15 for mc16 and cpuconsumptiontime
Doing train 3/15 for mc16 and cpuconsumptiontime
Doing train 4/15 for mc16 and cpuconsumptiontime
Doing train 5/15 for mc16 and cpuconsumptiontime
Doing train 6/15 for mc16 and cpuconsumptiontime
Doing train 7/15 for mc16 and cpuconsumptiontime
Doing train 8/15 for mc16 and cpuconsumptiontime
Doing train 9/15 for mc16 and cpuconsumptiontime
Doing train 10/15 for mc16 and cpuconsumptiontime
Doing train 11/15 for mc16 and cpuconsumptiontime
Doing train 12/15 for mc16 and cpuconsumptiontime
Doing train 13/15 for mc16 and cpuconsumptiontime
Doing train 14/15 for mc16 and cpuconsumptiontime
Doing train 15/15 for mc16 and cpuconsumptiontime
Doing train 1/1 for BPHYSLS and cpuconsumptiontime
Doing train 1/15 for data18 and cpuconsumptiontime
Doing train 2/15 for data18 and cpuconsumptiontime
Doing train 3/15 for data18 and cpuconsumptiontime
Doing train 4/15 for data18 and cpuconsumptiontime


/usr/lib64/python3.4/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Doing train 5/15 for data18 and cpuconsumptiontime
Doing train 6/15 for data18 and cpuconsumptiontime
Doing train 7/15 for data18 and cpuconsumptiontime
Doing train 8/15 for data18 and cpuconsumptiontime
Doing train 9/15 for data18 and cpuconsumptiontime
Doing train 10/15 for data18 and cpuconsumptiontime
Doing train 11/15 for data18 and cpuconsumptiontime
Doing train 12/15 for data18 and cpuconsumptiontime
Doing train 13/15 for data18 and cpuconsumptiontime
Doing train 14/15 for data18 and cpuconsumptiontime
Doing train 15/15 for data18 and cpuconsumptiontime
Doing train 1/1 for DELAYED and cpuconsumptiontime
nevents
Doing train 1/1 for mc15 and nevents


/usr/lib64/python3.4/site-packages/pandas/plotting/_core.py:1716: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  series.name = label
/usr/lib64/python3.4/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Doing train 1/15 for mc16 and nevents
Doing train 2/15 for mc16 and nevents
Doing train 3/15 for mc16 and nevents
Doing train 4/15 for mc16 and nevents
Doing train 5/15 for mc16 and nevents
Doing train 6/15 for mc16 and nevents
Doing train 7/15 for mc16 and nevents
Doing train 8/15 for mc16 and nevents
Doing train 9/15 for mc16 and nevents
Doing train 10/15 for mc16 and nevents
Doing train 11/15 for mc16 and nevents
Doing train 12/15 for mc16 and nevents
Doing train 13/15 for mc16 and nevents
Doing train 14/15 for mc16 and nevents
Doing train 15/15 for mc16 and nevents
Doing train 1/1 for BPHYSLS and nevents
Doing train 1/15 for data18 and nevents
Doing train 2/15 for data18 and nevents
Doing train 3/15 for data18 and nevents
Doing train 4/15 for data18 and nevents
Doing train 5/15 for data18 and nevents
Doing train 6/15 for data18 and nevents
Doing train 7/15 for data18 and nevents
Doing train 8/15 for data18 and nevents
Doing train 9/15 for data18 and nevents
Doing train 10/15 for da

KeyError: 'HION'

In [ ]:
#from matplotlib.pyplot import text
##execfile('./label_lines.ipynb')
#scope = "data17"
#grp = "SUSY"
#toplot["data17"] = pd.read_csv("data/maxpss_timeseries_data17.csv",index_col='date',parse_dates=['date'])
#toplot["data17"].index.values
#ax = toplot[scope].plot(y=group_trains[scope][grp],figsize=(40,15),legend=False,marker='o',
#                        title="Group: %s, Scope: %s" %(grp,scope),colormap="rainbow",markersize=25)
#ax.xaxis_date()
##plt.ylabel('maxvmem [MB]')
#if not 'train' in grp: ax.legend(bbox_to_anchor=(1.01, 1.05))
#else: ax.legend(loc=9, bbox_to_anchor=(0.5, -0.2),ncol=3)
##if 'cpuconsumptiontime' in metric: 
##    ax.yaxis.set_major_locator(HourLocator())
##    ax.yaxis.set_major_formatter(DateFormatter('%H:%M'))
##    plt.gca().yaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M')) 
#ax.set_xticks(np.array(metrics[metric][scope]['date'], dtype=object))
#plt.ylabel(ylabels[metric])
#plt.xlabel("ART date")

In [ ]:
#plt.figure(figsize=(40,15))
#nval = 0
#values = list((toplot[scope].loc[:,group_trains[scope][grp]]).columns.values)
#print(values[0])
#ax = toplot[scope].plot(y=values[0],figsize=(40,15),legend=False,marker='o',title="Group: %s, Scope: %s" %(grp,scope),colormap="rainbow")
#
#for val in values:
#    if val == 0:
#        nval += 1
#        continue
#    else:
#        toplot[scope].plot(y=group_trains[scope][grp],figsize=(40,15),legend=False,marker='o',markersize=20,
#                            title="Group: %s, Scope: %s" %(grp,scope),colormap="rainbow",ax=ax)
#    #plt.plot(toplot[scope].index.values,toplot[scope].loc[:,val],marker='o',linestyle='dashed',markersize=10)
#    nval += 1
#plt.show()
#ax = toplot[scope].plot(y=group_trains[scope][grp],figsize=(40,15),legend=False,marker='o',
                        #title="Group: %s, Scope: %s" %(grp,scope),colormap="rainbow")
#ax.get_lines()[-1].get_label()
#labelLines(ax.get_lines(),ax)
#print("h")
#text(2,0,'\leftarrow sin(\pi)')
#line = ax.get_lines()
#print(line[0].get_xdata(False))
##list(ax.get_xticklabels())
#ticks = [t for t in ax.get_xticklabels()]
#for i, t in enumerate(ticks):
#    print("Label {}, data: {}".format(i, t.get_text()), t.get_window_extent())
#print(line)

In [58]:
def color_red_or_green(val):
    color = 'green' if val == 'finished' else 'red'
    return 'background-color: %s' % color

In [65]:
import copy
toplot = {}
for scope in trainsAndIDs.keys():
    toplot[scope] = pd.read_csv("data/%s_timeseries_%s.csv"%("jobstatus",scope),index_col='date')
    print("Scope: %s:"%scope)
    #print(toplot[scope])
    if scope == "mc16": pattern_str = ""
    for grp in group_trains[scope]:
        md = ((((toplot[scope].loc[:,group_trains[scope][grp]]).transpose()).iloc[:, ::-1]))
        #md = ((((toplot[scope].loc[:,group_trains[scope][grp]])).iloc[:, ::-1]))
        #print((md))
        md = md.loc[:,~md.columns.duplicated()]
        st = (md.style.applymap(color_red_or_green)).render()
        #print("Saving figure to web/%s_%s_%s.html" %("status",scope,grp))
        with open("web/%s_%s_%s.html" %("status",scope,grp), 'w') as the_file:
            the_file.write(st)
        #print("grp")
        nmd = 0
        date_idx = 1
        for m in (md.iloc[:,date_idx]).index:
            #print(m)
            #print(md.iloc[:,date_idx][nmd])
            #print(scope)
            #print(len(m.split()))
            if len(m.split()) > 1 and not md.iloc[:,date_idx][nmd] == "finished":
                formats = m.split()
                new_train = copy.deepcopy(formats)
                #print("train is %s"%m)
                for f in formats:
                    #print(f)
                    #print(toplot[scope].loc[:,[f]].transpose())
                    try:
                        md2 = ((((toplot[scope].loc[:,[f]]).transpose()).iloc[:, ::-1]))
                    except:
                        print("Problems with format ",f)
                        continue
                    #print(md2)
                    if not md2.iloc[:,date_idx][0] == "finished":
                        new_train.remove(f)
                #print("remaining train is %s"%" ".join(new_train))
                if scope == "data18": # and len(new_train) > 0
                    pattern_str += "formats:DAOD_%s\n"%".DAOD_".join(new_train)
                    pattern_str += "ds:mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttbar_hdamp258p75_nonallhad.merge.AOD.e5458_s3126_r9781_r9778\n"
            elif md.iloc[:,date_idx][nmd] == "finished" and scope == "data18": #len(m.split()) > 1 and :
                #print("train is %s"%m)
                pattern_str += "formats:DAOD_%s\n"%".DAOD_".join(m.split())
                pattern_str += "ds:mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttbar_hdamp258p75_nonallhad.merge.AOD.e5458_s3126_r9781_r9778\n"
            elif not md.iloc[:,date_idx][nmd] == "finished":
                print("train is %s"%m)
                print("status is %s"%md.iloc[:,date_idx][nmd])
            #if md.iloc[:,0][nmd] != "finished": print("%-10s %-70s %-20s" %(scope,m,md.iloc[:,0][nmd]))
            nmd += 1
        #break
    #break

Scope: mc15:
Scope: mc16:
train is EXOT23
status is failed
train is SUSY7
status is failed
train is HIGG3D1
status is failed
train is TOPQ5
status is nan
train is BPHY14
status is nan
train is BPHY15
status is nan
train is BPHY16
status is nan
Scope: BPHYSLS:
Scope: data18:
train is PHYSVAL
status is failed
train is HIGG3D1
status is failed
train is SUSY7
status is failed
train is BPHY14
status is nan
train is BPHY15
status is nan
train is BPHY16
status is nan
Scope: DELAYED:


In [57]:
print(pattern_str)
print(trainsAndIDs.keys())

formats:DAOD_EXOT0
ds:mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttbar_hdamp258p75_nonallhad.merge.AOD.e5458_s3126_r9781_r9778
formats:DAOD_EXOT10
ds:mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttbar_hdamp258p75_nonallhad.merge.AOD.e5458_s3126_r9781_r9778
formats:DAOD_EXOT12
ds:mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttbar_hdamp258p75_nonallhad.merge.AOD.e5458_s3126_r9781_r9778
formats:DAOD_EXOT13
ds:mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttbar_hdamp258p75_nonallhad.merge.AOD.e5458_s3126_r9781_r9778
formats:DAOD_EXOT15
ds:mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttbar_hdamp258p75_nonallhad.merge.AOD.e5458_s3126_r9781_r9778
formats:DAOD_EXOT17
ds:mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttbar_hdamp258p75_nonallhad.merge.AOD.e5458_s3126_r9781_r9778
formats:DAOD_EXOT19
ds:mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttbar_hdamp258p75_nonallhad.merge.AOD.e5458_s3126_r9781_r9778
formats:DAOD_EXOT2
ds:mc16_13TeV.410501.PowhegPythia8EvtGen_A14_ttbar_hdamp258p75_nonallhad.merge.AOD.e5458_s312

These are the various tests which are run

In [30]:
group_trains.keys()

dict_keys(['mc15', 'mc16', 'BPHYSLS', 'data18', 'DELAYED'])

Sets the scope and it calculates the sizefrac (needs the filesizes extracted from one of the cells at the top)

In [31]:
scope = "data18"
sizefrac = pd.read_csv("data/outputfilebytes_timeseries_%s.csv"%scope,index_col='date',parse_dates=['date'])
insize = 1.0
sizefrac.loc[:,'EXOT23'] /= float(infilesizes['dataFileEXOT23'])
sizefrac.loc[:,group_trains[scope]['HION']] /= float(infilesizes['heavyIonFile'])
sizefrac[sizefrac.columns.difference(group_trains[scope]['HION']+['EXOT23'])] /= float(infilesizes['dataFile'])

KeyError: 'HION'

The one below gives you the latest size fraction for both trains and individual formats

In [ ]:
sizefractions_value  = []
sizefractions_format = []
lst = list(sizefrac.iloc[[-2]].items())
print(len(lst))
doBreak = False
while len(sizefractions_format) < len(lst) and not doBreak:
    max_sf = -999
    max_fm = ""
    for name, values in lst:
        # check if it is a train
        #print(name.split())
        if len(name.split()) > 1: continue 
        if values[0] > max_sf and not name in sizefractions_format:
            max_sf = values[0]
            max_fm = name
            #print('{name}: {value}'.format(name=name, value=values[0]))
    if max_fm in sizefractions_format or max_sf == -999:
        print("WARNING \t Key %s is already in keys. BREAK!" %max_fm)
        break
    sizefractions_value.append(max_sf)
    sizefractions_format.append(max_fm)
    #print("length = %i"%len(sizefractions.keys()))

The below prints a sorted list of the size fractions

In [ ]:
for i in range(len(sizefractions_value)):
    print("%20s %4.6f" %(sizefractions_format[i],sizefractions_value[i]*100.))

In [ ]:
float(infilesizes['dataFile'])/1.0e9
932014369./float(infilesizes['dataFile'])

<style  type="text/css" >
    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col0 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col1 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col2 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col3 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col4 {
            background-color:  red;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col5 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col6 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col7 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col8 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col9 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col10 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col11 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col12 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col13 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col14 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col15 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col16 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col17 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col18 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col19 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col20 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col21 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col22 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col23 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col24 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col25 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col26 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col27 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col28 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col0 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col1 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col2 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col3 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col4 {
            background-color:  red;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col5 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col6 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col7 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col8 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col9 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col10 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col11 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col12 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col13 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col14 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col15 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col16 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col17 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col18 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col19 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col20 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col21 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col22 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col23 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col24 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col25 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col26 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col27 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col28 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col0 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col1 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col2 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col3 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col4 {
            background-color:  red;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col5 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col6 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col7 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col8 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col9 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col10 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col11 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col12 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col13 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col14 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col15 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col16 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col17 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col18 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col19 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col20 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col21 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col22 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col23 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col24 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col25 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col26 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col27 {
            background-color:  green;
        }    #T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col28 {
            background-color:  green;
        }</style>  
<table id="T_d41a194e_1721_11e8_89ed_001b215b3eeb" > 
<thead>    <tr> 
        <th class="index_name level0" >date</th> 
        <th class="col_heading level0 col0" >2018-02-20 00:00:00</th> 
        <th class="col_heading level0 col1" >2018-02-19 00:00:00</th> 
        <th class="col_heading level0 col2" >2018-02-18 00:00:00</th> 
        <th class="col_heading level0 col3" >2018-02-17 00:00:00</th> 
        <th class="col_heading level0 col4" >2018-02-16 00:00:00</th> 
        <th class="col_heading level0 col5" >2018-02-15 00:00:00</th> 
        <th class="col_heading level0 col6" >2018-02-14 00:00:00</th> 
        <th class="col_heading level0 col7" >2018-02-13 00:00:00</th> 
        <th class="col_heading level0 col8" >2018-02-12 00:00:00</th> 
        <th class="col_heading level0 col9" >2018-02-11 00:00:00</th> 
        <th class="col_heading level0 col10" >2018-02-10 00:00:00</th> 
        <th class="col_heading level0 col11" >2018-02-09 00:00:00</th> 
        <th class="col_heading level0 col12" >2018-02-08 00:00:00</th> 
        <th class="col_heading level0 col13" >2018-02-07 00:00:00</th> 
        <th class="col_heading level0 col14" >2018-02-06 00:00:00</th> 
        <th class="col_heading level0 col15" >2018-02-04 00:00:00</th> 
        <th class="col_heading level0 col16" >2018-02-03 00:00:00</th> 
        <th class="col_heading level0 col17" >2018-02-02 00:00:00</th> 
        <th class="col_heading level0 col18" >2018-02-01 00:00:00</th> 
        <th class="col_heading level0 col19" >2018-01-31 00:00:00</th> 
        <th class="col_heading level0 col20" >2018-01-30 00:00:00</th> 
        <th class="col_heading level0 col21" >2018-01-29 00:00:00</th> 
        <th class="col_heading level0 col22" >2018-01-28 00:00:00</th> 
        <th class="col_heading level0 col23" >2018-01-27 00:00:00</th> 
        <th class="col_heading level0 col24" >2018-01-26 00:00:00</th> 
        <th class="col_heading level0 col25" >2018-01-25 00:00:00</th> 
        <th class="col_heading level0 col26" >2018-01-24 00:00:00</th> 
        <th class="col_heading level0 col27" >2018-01-23 00:00:00</th> 
        <th class="col_heading level0 col28" >2018-01-22 00:00:00</th> 
    </tr></thead> 
<tbody>    <tr> 
        <th id="T_d41a194e_1721_11e8_89ed_001b215b3eeblevel0_row0" class="row_heading level0 row0" >TRUTH0</th> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col0" class="data row0 col0" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col1" class="data row0 col1" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col2" class="data row0 col2" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col3" class="data row0 col3" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col4" class="data row0 col4" >nan</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col5" class="data row0 col5" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col6" class="data row0 col6" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col7" class="data row0 col7" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col8" class="data row0 col8" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col9" class="data row0 col9" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col10" class="data row0 col10" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col11" class="data row0 col11" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col12" class="data row0 col12" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col13" class="data row0 col13" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col14" class="data row0 col14" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col15" class="data row0 col15" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col16" class="data row0 col16" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col17" class="data row0 col17" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col18" class="data row0 col18" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col19" class="data row0 col19" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col20" class="data row0 col20" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col21" class="data row0 col21" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col22" class="data row0 col22" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col23" class="data row0 col23" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col24" class="data row0 col24" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col25" class="data row0 col25" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col26" class="data row0 col26" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col27" class="data row0 col27" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow0_col28" class="data row0 col28" >finished</td> 
    </tr>    <tr> 
        <th id="T_d41a194e_1721_11e8_89ed_001b215b3eeblevel0_row1" class="row_heading level0 row1" >TRUTH1</th> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col0" class="data row1 col0" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col1" class="data row1 col1" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col2" class="data row1 col2" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col3" class="data row1 col3" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col4" class="data row1 col4" >nan</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col5" class="data row1 col5" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col6" class="data row1 col6" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col7" class="data row1 col7" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col8" class="data row1 col8" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col9" class="data row1 col9" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col10" class="data row1 col10" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col11" class="data row1 col11" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col12" class="data row1 col12" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col13" class="data row1 col13" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col14" class="data row1 col14" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col15" class="data row1 col15" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col16" class="data row1 col16" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col17" class="data row1 col17" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col18" class="data row1 col18" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col19" class="data row1 col19" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col20" class="data row1 col20" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col21" class="data row1 col21" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col22" class="data row1 col22" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col23" class="data row1 col23" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col24" class="data row1 col24" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col25" class="data row1 col25" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col26" class="data row1 col26" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col27" class="data row1 col27" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow1_col28" class="data row1 col28" >finished</td> 
    </tr>    <tr> 
        <th id="T_d41a194e_1721_11e8_89ed_001b215b3eeblevel0_row2" class="row_heading level0 row2" >TRUTH3</th> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col0" class="data row2 col0" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col1" class="data row2 col1" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col2" class="data row2 col2" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col3" class="data row2 col3" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col4" class="data row2 col4" >nan</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col5" class="data row2 col5" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col6" class="data row2 col6" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col7" class="data row2 col7" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col8" class="data row2 col8" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col9" class="data row2 col9" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col10" class="data row2 col10" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col11" class="data row2 col11" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col12" class="data row2 col12" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col13" class="data row2 col13" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col14" class="data row2 col14" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col15" class="data row2 col15" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col16" class="data row2 col16" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col17" class="data row2 col17" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col18" class="data row2 col18" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col19" class="data row2 col19" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col20" class="data row2 col20" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col21" class="data row2 col21" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col22" class="data row2 col22" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col23" class="data row2 col23" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col24" class="data row2 col24" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col25" class="data row2 col25" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col26" class="data row2 col26" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col27" class="data row2 col27" >finished</td> 
        <td id="T_d41a194e_1721_11e8_89ed_001b215b3eebrow2_col28" class="data row2 col28" >finished</td> 
    </tr></tbody> 
</table> 